In [4]:
import numpy as np
import matplotlib.pyplot as plt

import math

import cv2

In [15]:
def estimate_intrinsics_linear(points_2d, points_3d, R, t):
    """
    Estimates intrinsic camera parameters (fx, fy, cx, cy) using a linear least-squares approach (corrected).

    Args:
        points_2d (numpy.ndarray): Array of 2D pixel coordinates (N x 2), where N >= 2.
        points_3d (numpy.ndarray): Array of corresponding 3D world coordinates (N x 3).
        R (numpy.ndarray): 3x3 rotation matrix.
        t (numpy.ndarray): 3x1 translation vector.

    Returns:
        tuple: Estimated fx, fy, cx, cy. Returns None if the input is invalid.
    """
    if points_2d.shape[0] != points_3d.shape[0] or points_2d.shape[0] < 2:
        print("Error: Number of 2D and 3D points must be the same and at least 2.")
        return None

    num_points = points_2d.shape[0]
    A = np.zeros((2 * num_points, 4))
    b = np.zeros(2 * num_points)

    Rt = np.hstack((R, t.reshape(-1, 1)))

    for i in range(num_points):
        u, v = points_2d[i]
        Xw, Yw, Zw = points_3d[i]
        Pw_homogenous = np.array([Xw, Yw, Zw, 1])
        Pc_homogenous = Rt @ Pw_homogenous
        Xc, Yc, Zc = Pc_homogenous[:3]

        A[2 * i, :] = [Xc, 0, Zc, 0]
        b[2 * i] = u * Zc

        A[2 * i + 1, :] = [0, Yc, 0, Zc]
        b[2 * i + 1] = v * Zc

    # Solve using least-squares
    try:
        x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)
        fx = x[0]
        fy = x[1]
        cx = x[2]
        cy = x[3]
        return fx, fy, cx, cy
    except np.linalg.LinAlgError:
        print("Error: Singular matrix encountered. Cannot solve for intrinsics.")
        return None

In [44]:
L,W,H = (35000, 18000, 2000)
theta = 0
val_cos = np.cos(np.deg2rad(theta))
val_sin = np.sin(np.deg2rad(theta))
R_l = np.array([
    [val_cos,0,val_sin],
    [0,1,0],
    [-val_sin,0,val_cos],
])

R_r = np.array([
    [val_cos,0,-val_sin],
    [0,1,0],
    [val_sin,0,val_cos],
])

t = np.array([[L/2], [H], [0]])

fx = 504.02353288
fy = 426.06999938
cx = 505.68393343
cy = 240.31930792
K = np.array([
    [fx, 0, cx],
    [0, fy, cy],
    [0, 0, 1]
])

In [45]:
point_pairs_cam1 = np.array([
    [500, 2100, 0,0,0],
    [2090,2010, 0,0,W],
    [3600,2200, L/2,0,W],
    [3610,2400, L/2,0,W/2],
])

point_pairs_cam2 = np.array([
    [3410,1990, L,0,0],
    [1900,1850, L,0,W],
    [386,1990, L/2,0,W],
    [350,2225, L/2,0,W/2],
])

# Separate 2D and 3D points for each camera
points_2d_cam1 = point_pairs_cam1[:, :2]
points_3d_cam1 = point_pairs_cam1[:, 2:]

points_2d_cam2 = point_pairs_cam2[:, :2]
points_3d_cam2 = point_pairs_cam2[:, 2:]

# Estimate intrinsics for Camera 1
intrinsics_cam1 = estimate_intrinsics_linear(points_2d_cam1, points_3d_cam1, R_r, t)
if intrinsics_cam1:
    fx1, fy1, cx1, cy1 = intrinsics_cam1
    print("Estimated intrinsics for Camera 1:")
    print(f"fx: {fx1:.2f}, fy: {fy1:.2f}, cx: {cx1:.2f}, cy: {cy1:.2f}")

print("\n")

# Estimate intrinsics for Camera 2
intrinsics_cam2 = estimate_intrinsics_linear(points_2d_cam2, points_3d_cam2, R_l, t)
if intrinsics_cam2:
    fx2, fy2, cx2, cy2 = intrinsics_cam2
    print("Estimated intrinsics for Camera 2:")
    print(f"fx: {fx2:.2f}, fy: {fy2:.2f}, cx: {cx2:.2f}, cy: {cy2:.2f}")

Estimated intrinsics for Camera 1:
fx: 420.13, fy: 482.73, cx: 2203.85, cy: 2078.18


Estimated intrinsics for Camera 2:
fx: 50.67, fy: 499.09, cx: 923.51, cy: 1892.27


In [46]:
def project_points(points_3d, R, t, K):
    points_3d_cam = R @ points_3d.T + t  # Apply extrinsic transform
    points_2d = K @ points_3d_cam        # Apply intrinsics
    points_2d /= points_2d[2]            # Normalize
    return points_2d[:2].T

Projected 2D Points (Right Cam):
 [[          inf           inf]
 [          inf           inf]
 [1485.72969181  287.66041896]
 [2465.77545019  335.00153   ]]


/tmp/ipykernel_223229/778094543.py:4: RuntimeWarning: divide by zero encountered in divide
  points_2d /= points_2d[2]            # Normalize
/tmp/ipykernel_223229/778094543.py:4: RuntimeWarning: invalid value encountered in divide
  points_2d /= points_2d[2]            # Normalize


In [ ]:
pts1_norm = cv2.undistortPoints(pts1.reshape(-1,1,2), K, None).reshape(-1,2)
pts2_norm = cv2.undistortPoints(pts2.reshape(-1,1,2), K, None).reshape(-1,2)

# Estimate Essential Matrix from normalized points
E, _ = cv2.findEssentialMat(pts1_norm, pts2_norm, focal=1.0, pp=(0.,0.), method=cv2.RANSAC, prob=0.999, threshold=1e-3)

# Decompose E to get possible R, t (you already have t, this estimates R)
_, R_est, _, _ = cv2.recoverPose(E, pts1, pts2, K)